# Objetos geradores

## 1. Estrutura básica de um gerador

Vimos anteriormente que Python usa _geradores_ em algumas situações, e vimos duas formas de definir geradores:

- A expressão geradora, do tipo:
  ```
  (i ** 3 for i in range(100))
  ```
- A função geradora, do tipo:
  ```
  def pow3():
      for i in range(100):
          yield i ** 3
  ```

Outra possibilidade é definir os objetos da sua classe como geradores, isto é, objetos que permitem acesso de um valor por vez de um conjunto de valores.

Isso é feito definindo os métodos `__iter__` e `__next__`. O método `__iter__` é chamado quando o objeto é usado em um contexto onde o Python espera um iterador (por exemplo, em um `for`). Em muitos casos, basta retornar o próprio objeto (mais sobre isso adiante). O método `__next__` é chamado quando o programa precisa do próximo valor. O objeto sinaliza que todos os valores já foram fornecidos fazendo um `raise` de `StopIteration()`.

Objetos da classe abaixo geram valores consecutivos similaremente a `range(ini, fin)`:

In [ ]:
class MyRange:
    def __init__(self, ini, fin):
        self._ini = ini
        self._fin = fin
        self._current = ini
        
    def __next__(self):
        if self._current == self._fin:
            raise StopIteration()
        val = self._current
        self._current += 1
        return val
        
    def __iter__(self):
        return self

Note que os métodos não usam `yield`, e sim `return`. O fato de o objeto ser um gerador é determinado pela presença dos métodos mágicos `__iter__` e `__next__`.

In [ ]:
m = MyRange(0,10)
for x in m:
    print(x)

Os métodos `__iter__` e `__next__` podem ser chamados diretamente usando as funções `iter` e `next`, respectivamente.

In [ ]:
m = MyRange(0,10)

In [ ]:
mi = iter(m)

Neste caso, a chamada acima é desnecessária, pois o `iter` irá retornar o próprio objeto `m`, mas a chamada é necessária no caso geral, como veremos adiante.

In [ ]:
next(mi)

In [ ]:
next(mi)

In [ ]:
next(mi)

In [ ]:
next(mi)

Note como o `next` opera sobre o objeto retornado pelo `iter`.

In [ ]:
for i in range(5):
    next(mi)

In [ ]:
next(mi)

In [ ]:
next(mi)

In [ ]:
[2 * x - 3 for x in MyRange(10, 15)]

## 2. Geradores para outros objetos

Conforme definido, um iterador, após exaurido, não pode mais ser lido. Quer dizer, temos acesso aos valores do iterador apenas uma vez.

In [ ]:
m = MyRange(0, 10)

In [ ]:
for x in m:
    print(x)

In [ ]:
next(m)

Como `m` já foi exaurido, um for por ele termina imediatamente.

In [ ]:
for y in m:
    print(y)

Um resultado disso é que não podemos usar o gerador em mais do que um lugar ao mesmo tempo. Por exemplo, o *loop* duplo abaixo não funciona como esperado a primeira vista:

In [ ]:
m = MyRange(1, 5)
for x in m:
    for y in m:
        print((x, y), end=' ')

Se quisermos usar os valores mais do que uma vez, uma opção é usar os valores para criar uma lista, que pode então ser percorrida múltiplas vezes.

In [ ]:
m = list(MyRange(1, 5))
for x in m:
    for y in m:
        print((x, y),end=' ')

Mas isto pode representar um desperdício de memória para listas grandes de valores. Uma outra opção é mudar a definição da classe, de forma que ao se chamar o método `__iter__` seja retornado um objeto de outra classe, que é o responsável por acompanhar os valores atuais e implementar `__next__`. Vamos refazer nosso `MyRange` dessa forma:

In [ ]:
# Esta classe controla quais valores já foram varridos.
class MyRangeIterator:
    def __init__(self, rg):
        self._range = rg
        self._current = rg._ini
        
    def __next__(self):
        if self._current == self._range._fin:
            raise StopIteration()
        val = self._current
        self._current += 1
        return val

# Esta classe controla os valores que pertencem à faixa desejada
class MyRange:
    def __init__(self, ini, fin):
        self._ini = ini
        self._fin = fin
        
    def __iter__(self):
        return MyRangeIterator(self)

Note como o método `__iter__` é implementado na classe da qual queremos percorrer os valores, e apenas retorna um objeto de um tipo auxiliar que é usado para percorrer os valores.  Na classe auxiliar, implementamos o método `__next__`.

Assim, neste caso, quando chamamos `iter(x)`, onde `x` é do tipo `MyRange`, ele irá retornar um objeto `MyRangeIterator`, que irá percorrer a faixa de valores do começo. Já a função `next` será chamada sobre esse objeto do tipo auxiliar para pegar cada valor por vez.

In [ ]:
m = MyRange(1, 5)

In [ ]:
mi = iter(m)

In [ ]:
type(m), type(mi)

In [ ]:
next(mi)

In [ ]:
next(mi)

In [ ]:
next(mi), next(mi)

In [ ]:
next(mi)

Como explicado antes, ao usarmos um objeto em um lugar que o Python espera um gerador (como num `for`), sera executado `iter` sobre esse objeto, para pegar seu iterador:

In [ ]:
for x in m:
    print(x, end=' ')

In [ ]:
for x in m:
    print(x**2)

Agora esse gerador pode ser varrido várias vezes, cada varredura sendo independente, pois em cada um dos `for` será executada uma chamada para `__iter__`.

In [ ]:
for x in m:
    for y in m:
        print((x,y), end=' ')

## 3. Funções geradoras para o método `__iter__`

Uma forma mais fácil de possibilitar múltiplas varreduras do gerador, útil em diversas situações, e que não necessita de definição de classes auxiliares é simplesmente retornar uma função geradora no método `__iter__`. Como funções geradoras implementam o método `__next__`, o código abaixo funciona:

In [ ]:
class MyRange:
    def __init__(self, ini, fin):
        self._ini = ini
        self._fin = fin
        
    def __iter__(self):
        for i in range(self._ini, self._fin):
            yield i

Note que nesse caso, quando `__iter__` for chamado (por exemplo, ao começar um `for`), ele irá retornar um objeto gerador, sobre o qual será executada `__next__` para cada iteração, resultando nos resultados retornados por `yield`.

In [ ]:
m = MyRange(1, 5)

In [ ]:
type(m)

In [ ]:
mi = iter(m)
type(mi)

Cada nova chamada de `__iter__` para uma mesma `MyRange` resulta no retorno de um novo objeto gerador, com os valores inicial e final fixados, e independentes de outras funções retornadas pela mesma chamada.

In [ ]:
for x in m:
    for y in m:
        print((x,y), end=' ')

## 4. Geradores no operador `in`

Uma outra situação onde os métodos `__iter__` e `__next__` da classe são usados é quando o objeto é usado à direita do operador `in`. Nesse caso, o Python chama `__iter__`, pega o objeto retornado e sobre ele vai chamando `__next__` e comparando com o valor à esquerda do `in`. Se algum valor retornado pelos `__next__` for igual, ele resulta em `True`; se a iteração terminar sem um valor igual, ele resulta em `False`.

In [ ]:
3 in m

In [ ]:
7 in m

Isso é uma funcionalidade interessante, mas para certos geradores implica numa quantidade muito grande de operações (comparar muitos valores). No nosso iterador, por exemplo, podemos saber se o valor está incluido na faixa apenas comparando com o menor e maior valores da faixa.

Se quisermos usar uma forma mais eficaz de implementar o operador `in`, podemos definir o método `__contains__` na classe.

In [ ]:
class MyRange:
    def __init__(self, ini, fin):
        self._ini = ini
        self._fin = fin
        
    def __iter__(self):
        for i in range(self._ini, self._fin):
            yield i
            
    def __contains__(self, val):
        return self._ini <= val < self._fin

In [ ]:
m = MyRange(1,5)

In [ ]:
3 in m

In [ ]:
11 in m

In [ ]:
large = MyRange(0, 100000)

In [ ]:
%timeit 1000000 in large

In [ ]:
%timeit 1000000 in large

# Exercício

Qual a saída produzida pelo código abaixo:
```python
class Pot10:
    def __init__(self, n):
        self._n = n
        self._next = 1
        self._i = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self._i == self._n:
            raise StopIteration()
        current = self._next
        self._next *= 10
        self._i += 1
        return current
    
tens = Pot10(10)
print('First half:')
for _, pt in zip(range(5), tens):
    print(pt)
print('Second half:')
for pt in tens:
    print(pt)
```